In [1]:
import numpy as np
import pandas as pd
import re

import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest

C:\Users\smvan\anaconda3\lib\site-packages\pdpipe\col_generation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
#consolidated_df = pd.read_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/Consolidated_purchases/OK_consol_purchases_march1_april21.csv")
# consolidated_df = pd.read_excel("C:/Users/smvan/CFFS-S23/CFFS-22-23/2023-07-31_New_Items.xlsx")
consolidated_df = pd.read_excel("C:/Users/smvan/CFFS-S23/CFFS-22-23/2023-12-22_New_Items.xlsx")

# consolidated_df = pd.read_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/mapping/new items/2023-07-20_New_Items.csv")
consolidated_df.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-11706,NaN,Glry Side Fries 2023,1.0,ea,1.0,ea,N
1,I-16780,NaN,2022 Vegan Caesar dressing,1.5,fl oz,1.0,fl oz,N
2,I-14715,NaN,2022 Coleslaw mix,7.0,oz,1.0,Kg,N
3,I-15427,NaN,Vegan Chipotle Mayo,2.0,fl oz,1.0,L,N
4,I-13956,NaN,Sushi Rice,6.0,PORT,6.0,PORT,N


In [3]:
# # Sort the dataframe by 'item_num' and 'qty'
# consolidated_df.sort_values(by=['item_num', 'qty'], inplace=True)

# # Find the rows to keep
# rows_to_keep = []
# prev_item_num = None

# for index, row in consolidated_df.iterrows():
#     if row['item_num'] != prev_item_num:
#         rows_to_keep.append(index)
#         prev_item_num = row['item_num']
#     elif row['qty'] < 0:
#         rows_to_keep.pop()
#         rows_to_keep.append(index)

# # Filter the dataframe to keep the selected rows
# unique_rows = consolidated_df.loc[rows_to_keep]

# # Display the unique rows
# unique_rows



In [4]:
# unique_rows[unique_rows['item_num'] == 69025]

In [5]:
categories_df = pd.read_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/GHG_categories/ghge_factors.csv")
categories_df


,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316
...,...,...,...
58,59,manually adjusted,0.0000
59,60,human labor,0.0000
60,61,kitchen supplies,0.0000
61,62,mushrooms,1.5000


In [6]:
consolidated_df.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-11706,NaN,Glry Side Fries 2023,1.0,ea,1.0,ea,N
1,I-16780,NaN,2022 Vegan Caesar dressing,1.5,fl oz,1.0,fl oz,N
2,I-14715,NaN,2022 Coleslaw mix,7.0,oz,1.0,Kg,N
3,I-15427,NaN,Vegan Chipotle Mayo,2.0,fl oz,1.0,L,N
4,I-13956,NaN,Sushi Rice,6.0,PORT,6.0,PORT,N


In [7]:
# Convert the "item_descrip" and "Food Category" columns to lowercase
consolidated_df['item_descrip'] = consolidated_df['Description'].str.lower()
categories_df['Food Category'] = categories_df['Food Category'].str.lower()

# Initialize new columns
consolidated_df['Food Category'] = np.nan

# Define a function to split the Food Category column
# def split_food_category(category):
#     separators = ['/', '(', ',', ')', '|']
#     for separator in separators:
#         category = category.replace(separator, ' ')
#     return category.strip()

def split_food_category(category):
    separators = ['/', '(', ',', ')', '|']
    for separator in separators:
        category = category.replace(separator, ' ')
    return category.strip()

for index, row in categories_df.iterrows():
    categories_df.at[index, 'Food Category'] = split_food_category(row['Food Category'])
    
for index, row in consolidated_df.iterrows():
    consolidated_df.at[index, 'item_descrip'] = split_food_category(row['item_descrip'])
    
    
# Iterate through each row
for index, row in consolidated_df.iterrows():
    item_descrip = row['item_descrip']
    
    # Split the item_descrip into words
    item_words = item_descrip.split()
    
    # Find matches between the words in item_descrip and the Food Category column
    matches = categories_df[categories_df['Food Category'].apply(lambda x: any(word in x.split() for word in item_words))]
    
    # If there is a match, assign the corresponding values
    if not matches.empty:
        consolidated_df.at[index, 'Food Category'] = matches['Food Category'].iloc[0]
        consolidated_df.at[index, 'CategoryID'] = matches['Category ID'].iloc[0]

# Print the updated DataFrame
consolidated_df





,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,item_descrip,Food Category
0,I-11706,NaN,Glry Side Fries 2023,1.0,ea,1.0,ea,N,glry side fries 2023,NaN
1,I-16780,NaN,2022 Vegan Caesar dressing,1.5,fl oz,1.0,fl oz,N,2022 vegan caesar dressing,NaN
2,I-14715,NaN,2022 Coleslaw mix,7.0,oz,1.0,Kg,N,2022 coleslaw mix,NaN
3,I-15427,NaN,Vegan Chipotle Mayo,2.0,fl oz,1.0,L,N,vegan chipotle mayo,NaN
4,I-13956,25.0,Sushi Rice,6.0,PORT,6.0,PORT,N,sushi rice,rice
...,...,...,...,...,...,...,...,...,...,...
98,I-13398,NaN,BIB Mix Sprite,1.0,L,18.5,L,Y,bib mix sprite,NaN
99,I-13955,NaN,Sushi Su,1.0,L,50.0,L,N,sushi su,NaN
100,I-13736,58.0,2019 Ginger Garlic Paste,60.0,g,500.0,Kg,N,2019 ginger garlic paste,sauces & paste
101,I-18348,NaN,Gremolata 2022,5.0,g,1.0,g,N,gremolata 2022,NaN


In [8]:
# for index, row in categories_df.iterrows():
#     for i, r in consolidated_df.iterrows():
#         if consolidated_df.loc[i, 'Food Category'].str.contains(categories_df.loc[index, 'Food Category']):
#             consolidated_df.loc[index, 'Food Category'] = categories_df.loc[index, 'Food Category']
#             consolidated_df.loc[index, 'Category ID'] = categories_df.loc[index, 'Category ID']

In [9]:
# def split_food_category(category):
#     separators = ['/', '(', ',', ')']
#     for separator in separators:
#         category = category.replace(separator, ' ')
#     return category.strip()

# for index, row in categories_df.iterrows():
#     categories_df.at[index, 'Food Category'] = split_food_category(row['Food Category'])

a_df = consolidated_df.iloc[730:760, :]


In [10]:
# Iterate through each row
for index, row in consolidated_df.iterrows():
    item_descrip = row['item_descrip']
    
    # Split the item_descrip into words
    item_words = item_descrip.split()
    
    if 'kale' in item_words or 'lettuce' in item_words or 'romaine' in item_words or 'salad' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'CategoryID'] = 40
        
    if 'base' in item_words and 'veg' in item_words or 'veg' in item_words or 'seaweed' in item_words or 'ck|g&g|salad|kalecaesarw/chick' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'CategoryID'] = 40
        
    if 'nori' in item_words or 'ck|g&g|salad|kale caesar.' in item_words or 'pickle' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'CategoryID'] = 40

        
    if 'spinach' in item_words or 'artichoke' in item_words or 'avocado' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'CategoryID'] = 40
        
    if 'cantaloupe' in item_words or 'papaya' in item_words or 'eggplant' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'CategoryID'] = 31
    
    if 'celery' in item_words or 'coleslaw' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'CategoryID'] = 40
    
    if 'sprouts' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'CategoryID'] = 40
    
    if 'milkettes' in item_words or ('milk' in item_words and '2%' in item_words) or 'milk' in item_words:
        consolidated_df.at[index, 'Food Category'] = "milk (cow's milk)"
        consolidated_df.at[index, 'CategoryID'] = 9
    
    
    if ('milk' in item_words and 'rice' in item_words):
        consolidated_df.at[index, 'Food Category'] = "rice milk"
        consolidated_df.at[index, 'CategoryID'] = 29

            
    if 'oat' in item_words or 'oatmeal' in item_words or 'oats' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'oats (oatmeal)'
        consolidated_df.at[index, 'CategoryID'] = 23

    if 'dress' in item_words or 'sauce' in item_words or 'jelly' in item_words or 'ketchup' in item_words or 'chutney' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sauces & paste'
        consolidated_df.at[index, 'CategoryID'] = 58

    if 'mayonaise*vegan' in item_words or 'mayonnaise' in item_words or 'mustard' in item_words or 'spread' in item_words or 'paste' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sauces & paste'
        consolidated_df.at[index, 'CategoryID'] = 58
        
        

    # Check if item_descrip contains "cereal"
    if 'cereal' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other grains/cereals'
        consolidated_df.at[index, 'CategoryID'] = 21
              

    if 'coconut' in item_words or 'apricot' in item_words or 'cucumber' in item_words or 'peppers' in item_words or 'salsa' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'CategoryID'] = 31 

    if 'grapes' in item_words or 'kiwi' in item_words or 'mangoes' in item_words or 'honeydew' in item_words or 'peach' in item_words or 'pears' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'CategoryID'] = 31 
    
    if 'squash' in item_words or 'watermelon' in item_words or 'zucchini' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'CategoryID'] = 31 
        
    if 'muffin' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'CategoryID'] = 24     
  
        
        
    if 'pie' in item_words or 'waffle' in item_words or 'pastry' in item_words or 'flour' in item_words or 'bun' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'CategoryID'] = 24
    
    if 'bar' in item_words or 'biscuits' in item_words or 'brownie' in item_words or 'cake' in item_words or 'cookie' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'CategoryID'] = 24
        
    
    if 'loaf' in item_words or 'pretzel' in item_words or 'scone' in item_words or 'tart' in item_words or 'tiramisu' in item_words or 'lady' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'CategoryID'] = 24

    if 'crouton' in item_words or 'ck|prep|granola' in item_words or 'noodles' in item_words or 'perogy' in item_words or 'pasta' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'CategoryID'] = 24
    
               
#     if 'co2' in item_words or 'soda' in item_words or 'baking' in item_words:
#         consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
#         consolidated_df.at[index, 'Category ID'] = 61
            
        
    if 'hashbrown' in item_words or 'potato*pompom' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'potatoes'
        consolidated_df.at[index, 'CategoryID'] = 41
    
    if 'dragon' in item_words or 'california' in item_words or 'shrimpmt' in item_words or 'prawn' in item_words or 'shrimp' in item_words or 'prawns' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'crustaceans (shrimp/prawns)'
        consolidated_df.at[index, 'CategoryID'] = 13


    if 'cumin' in item_words or 'cloves' in item_words or 'ground' in item_words or 'seasoning' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'CategoryID'] = 54
        
    if 'blend' in item_words or 'chili' in item_words or 'pwdr' in item_words or 'caper' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'CategoryID'] = 54
    
    if 'leaf' in item_words or 'chile' in item_words or 'paprika' in item_words or 'leaves' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'CategoryID'] = 54   
        
    if 'anise' in item_words or 'cardamom' in item_words or 'mix' in item_words or 'wasabi' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'CategoryID'] = 54 
        
        
    if 'onion' in item_words or 'leeks' in item_words or 'onions' in item_words or 'shallots' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'onions and leeks'
        consolidated_df.at[index, 'CategoryID'] = 39
                
    
    if 'pumpkin' in item_words or 'jackfruit' in item_words or 'pineapple' in item_words or 'raisin' in item_words or 'cherries' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'CategoryID'] = 31 
        
    if 'pumpkin' in item_words and 'kern' in item_words or 'quinoa' in item_words or 'sesame' in item_words or 'seed' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'tree nuts and seeds'
        consolidated_df.at[index, 'CategoryID'] = 26 
        

    if 'tomato' in item_words or 'tomatillos' in item_words or 'tomatoes' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'tomatoes'
        consolidated_df.at[index, 'CategoryID'] = 37
        
    if 'soybean' in item_words or 'tofu' in item_words or 'tempeh' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'soybeans/tofu'
        consolidated_df.at[index, 'CategoryID'] = 20
    
    if 'sake' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wine grapes (wine)'
        consolidated_df.at[index, 'CategoryID'] = 51
    
    if 'lentils' in item_words or 'ck|g&g|salad|medchickpea' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other legumes'
        consolidated_df.at[index, 'CategoryID'] = 16
    
    if 'chickpea.' in item_words or 'falafel' in item_words or ('patties' in item_words and 'vegan' in item_words):
        consolidated_df.at[index, 'Food Category'] = 'other legumes'
        consolidated_df.at[index, 'CategoryID'] = 16
    
    
    if 'tuna' in item_words or 'squid' in item_words or 'cod' in item_words or 'halibut' in item_words or 'salmon' in item_words or 'sal' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'fish (finfish)'
        consolidated_df.at[index, 'CategoryID'] = 12
    
    if 'raspberry' in item_words or 'blackberry' in item_words or 'blueberry' in item_words or 'strawberry' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'berries'
        consolidated_df.at[index, 'CategoryID'] = 34
        
    if 'raspberries' in item_words or 'blackberries' in item_words or 'blueberries' in item_words or 'strawberries' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'berries'
        consolidated_df.at[index, 'CategoryID'] = 34
    
    if 'samosa' in item_words or 'wrap' in item_words or 'sandwich' in item_words or 'soup' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'manually adjusted'
        consolidated_df.at[index, 'CategoryID'] = 59
        
    if 'coconut' in item_words and 'milk' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'manually adjusted'
        consolidated_df.at[index, 'CategoryID'] = 59
 
    if 'mandarin' in item_words or 'grapefruit' in item_words or 'lemons' in item_words or 'limes' in item_words or 'mandarins' in item_words or 'oranges' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'citrus fruit'
        consolidated_df.at[index, 'CategoryID'] = 35
        
    if 'fruit' in item_words or 'fruit.' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'citrus fruit'
        consolidated_df.at[index, 'CategoryID'] = 35
    
    if 'olive' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'olives (oil)'
        consolidated_df.at[index, 'CategoryID'] = 49
    
    if 'beets' in item_words or 'dill' in item_words or 'ginger' in item_words or 'garlic' in item_words or 'jicama' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'root vegetables'
        consolidated_df.at[index, 'CategoryID'] = 38
   
    if 'parsnip' in item_words or 'carrot' in item_words or 'carrots' in item_words or 'turnip' in item_words or 'tapioca' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'root vegetables'
        consolidated_df.at[index, 'CategoryID'] = 38
        
        
    if 'thyme' in item_words or 'herb' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'herbs'
        consolidated_df.at[index, 'CategoryID'] = 63    
        
    if 'pea' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'peas'
        consolidated_df.at[index, 'CategoryID'] = 18 
    
    if 'yogurt' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'yogurt'
        consolidated_df.at[index, 'CategoryID'] = 10
        
    
    if 'crepe' in item_words or 'croissant' in item_words or 'danish' in item_words or 'donut' in item_words or 'eclair' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'CategoryID'] = 24
    
        # Check if prod_group_descrip is "SUPPLY"
#     if row['prod_group_descrip'] == 'SUPPLY':
#         consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
#         consolidated_df.at[index, 'Category ID'] = 61
        
#     if row['prod_group_descrip'] == 'DISPOSABLES':
#         consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
#         consolidated_df.at[index, 'Category ID'] = 61

#     if row['prod_group_descrip'] == 'CLEANING':
#         consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
#         consolidated_df.at[index, 'Category ID'] = 61
        
    if row['InventoryGroup'] == 'BREAD':
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'CategoryID'] = 24   
    
    if row['InventoryGroup'] == 'BAKED GOODS':
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'CategoryID'] = 24
        
    if row['InventoryGroup'] == 'CANDIES SNACKS':
        consolidated_df.at[index, 'Food Category'] = 'sugars and sweeteners'
        consolidated_df.at[index, 'CategoryID'] = 43
    
    if row['InventoryGroup'] == 'BEVERAGE' and pd.isna(row['Food Category']):
        consolidated_df.at[index, 'Food Category'] = 'water & beverages'
        consolidated_df.at[index, 'CategoryID'] = 55
    
    if 'pringles' in item_words or 'pringle' in item_words or 'potato' in item_words or 'fries' in item_words or 'chip' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'potatoes'
        consolidated_df.at[index, 'CategoryID'] = 41
    
    if ('cream' in item_words and 'ice' in item_words) or 'magnum' in item_words or ('mint' in item_words and 'chip' in item_words):
        consolidated_df.at[index, 'Food Category'] = 'ice cream'
        consolidated_df.at[index, 'CategoryID'] = 7
    
    if ('base' in item_words and 'chick' in item_words) or ('thigh' in item_words) or ('wings' in item_words) or 'chicken' in item_words or 'turkey' in item_words or 'chick' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'poultry (chicken, turkey)'
        consolidated_df.at[index, 'CategoryID'] = 4
    
        # Check if item_descrip contains "proscuitto" or "ham"
    if 'prosciutto' in item_words or 'ham' in item_words or 'bacon' in item_words or 'chorizo' in item_words or 'sausage' in item_words or 'pork' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'pork (pig meat)'
        consolidated_df.at[index, 'CategoryID'] = 3

    if 'juice' in item_words or 'pop' in item_words or 'coke' in item_words or 'ale' in item_words or 'inst' in item_words or 'tea' in item_words or 'bev' in item_words or 'punch' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'water & beverages'
        consolidated_df.at[index, 'CategoryID'] = 55 
        
    if 'water' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'water & beverages'
        consolidated_df.at[index, 'CategoryID'] = 55 
 
    if 'rubbed' in item_words or 'pepper' in item_words or 'powder' in item_words or 'masala' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'CategoryID'] = 54
        
    if row['InventoryGroup'] == 'SUPPLY':
        consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
        consolidated_df.at[index, 'CategoryID'] = 61
        
    if row['InventoryGroup'] == 'DISPOSABLES':
        consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
        consolidated_df.at[index, 'CategoryID'] = 61

    if row['InventoryGroup'] == 'CLEANING':
        consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
        consolidated_df.at[index, 'CategoryID'] = 61
        
    
    if 'beer' in item_words:
        consolidated_df.at[index, 'Food Category'] = "barley (beer)"
        consolidated_df.at[index, 'CategoryID'] = 50
    
    if 'sugar' in item_words or 'melona' in item_words or 'honey' in item_words or 'syrup' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sugars and sweeteners'
        consolidated_df.at[index, 'CategoryID'] = 43 
        

    if 'salt&vin' in item_words or 'balsamic' in item_words or 'vinegar' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'vinegar'
        consolidated_df.at[index, 'CategoryID'] = 57
    
    if 'salt' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'salt'
        consolidated_df.at[index, 'CategoryID'] = 56
        
    if 'butter' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'butter'
        consolidated_df.at[index, 'CategoryID'] = 5
          
        
    if ('cream' in item_words or 'creamo10%' in item_words) and not 'ice' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cream'
        consolidated_df.at[index, 'CategoryID'] = 8  
        
    if 'paneer' in item_words or 'cheese' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cheese'
        consolidated_df.at[index, 'CategoryID'] = 6
    
    
    if 'egg' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'eggs'
        consolidated_df.at[index, 'CategoryID'] = 11
        
    
    if ('milk' in item_words and 'go' in item_words) or ('milk' in item_words and '2%' in item_words) or ('milk' in item_words and 'choc' in item_words):
        consolidated_df.at[index, 'Food Category'] = "milk (cow's milk)"
        consolidated_df.at[index, 'CategoryID'] = 9
        
    if 'milk' in item_words and 'homo' in item_words:
        consolidated_df.at[index, 'Food Category'] = "milk (cow's milk)"
        consolidated_df.at[index, 'CategoryID'] = 9

    if ('milk' in item_words and 'soy' in item_words):
        consolidated_df.at[index, 'Food Category'] = "soy milk"
        consolidated_df.at[index, 'CategoryID'] = 30
        
    if ('milk' in item_words and 'oat' in item_words) or ('oat' in item_words and 'barista' in item_words):
        consolidated_df.at[index, 'Food Category'] = "oat milk"
        consolidated_df.at[index, 'CategoryID'] = 28
    
    if 'banana' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'bananas'
        consolidated_df.at[index, 'CategoryID'] = 33
    
    if 'mushrooms' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'mushrooms'
        consolidated_df.at[index, 'CategoryID'] = 62
        
    if 'corn' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'corn (maize)'
        consolidated_df.at[index, 'CategoryID'] = 22
    
    if 'lamb' in item_words or 'mutton' in item_words or 'goat' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'lamb/mutton & goat meat'
        consolidated_df.at[index, 'CategoryID'] = 2
        
        
    if 'broccoli' in item_words or 'cabbage' in item_words or 'cauliflower' in item_words or 'kale' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cabbages and other brassicas (broccoli)'
        consolidated_df.at[index, 'CategoryID'] = 36   

    if 'daikon' in item_words or 'brussel' in item_words or 'choy' in item_words or 'broccolini' in item_words or 'gai' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cabbages and other brassicas (broccoli)'
        consolidated_df.at[index, 'CategoryID'] = 36 

    if 'co2' in item_words or 'soda' in item_words or 'baking' in item_words or 'sponge' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
        consolidated_df.at[index, 'CategoryID'] = 61
    
    if 'easter' in item_words or 'candy' in item_words or 'sweetener' in item_words or 'puddings' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sugars and sweeteners'
        consolidated_df.at[index, 'CategoryID'] = 43
        
    if 'pumpkin' in item_words or 'hemp' in item_words or 'nut' in item_words or 'almond' in item_words or 'seeds' in item_words or 'chia' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'tree nuts and seeds'
        consolidated_df.at[index, 'CategoryID'] = 26
        
    if 'oil' in item_words or 'oils' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetable oils'
        consolidated_df.at[index, 'CategoryID'] = 44

    if ('oil' in item_words and 'soybeans' in item_words) or ('oil' in item_words and 'soybean' in item_words) or ('oil' in item_words and 'soy' in item_words):
        consolidated_df.at[index, 'Food Category'] = 'soybeans (oil)'
        consolidated_df.at[index, 'CategoryID'] = 45
    
    if 'oil' in item_words and 'palm' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'palm (oil)'
        consolidated_df.at[index, 'CategoryID'] = 46

    if 'oil' in item_words and 'sunflower' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sunflower (oil)'
        consolidated_df.at[index, 'CategoryID'] = 47

    if ('oil' in item_words and 'rapeseed' in item_words) or ('oil' in item_words and 'canola' in item_words):
        consolidated_df.at[index, 'Food Category'] = 'rapeseed/canola (oil)'
        consolidated_df.at[index, 'CategoryID'] = 48
        
    if 'bean' in item_words or 'beans' in item_words or 'pulses' in item_words or 'chana' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'beans and pulses (dried)'
        consolidated_df.at[index, 'CategoryID'] = 17
    
    
    if 'chocolate' in item_words or 'arnotts' in item_words or 'choco' in item_words or 'cho' in item_words or 'cocoa' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cocoa'
        consolidated_df.at[index, 'CategoryID'] = 52
        
    if 'dough' in item_words or 'cracker' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'CategoryID'] = 24
    
    if 'rice' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'rice'
        consolidated_df.at[index, 'CategoryID'] = 25
        
    if 'apples' in item_words or 'apple' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'apples'
        consolidated_df.at[index, 'CategoryID'] = 32  
    
    if ('ice' in item_words and 'cream' in item_words) or 'melona' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'ice cream'
        consolidated_df.at[index, 'CategoryID'] = 7
    
        # Check if item_descrip contains "salami"
    if 'salami' in item_words or '50/50' in item_words or 'patties' in item_words or ('beef' in item_words and 'roast' in item_words) or 'beef' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'beef & buffalo meat'
        consolidated_df.at[index, 'CategoryID'] = 1
        
    if 'ht' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'ice cream'
        consolidated_df.at[index, 'CategoryID'] = 7
        
    if 'oyster' in item_words or 'oysters' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'mollusks'
        consolidated_df.at[index, 'CategoryID'] = 14
    
# consolidated_df
# 18386

#     elif 'chip' in item_words or 'potato' in item_words:
#         consolidated_df.at[index, 'Food Category'] = 'potatoes'
#         consolidated_df.at[index, 'Category ID'] = 41
# Print the updated DataFrame

In [11]:
# def split_food_category(category):
#     separators = ['/', '(', ',', ')']
#     for separator in separators:
#         category = category.replace(separator, ' ')
#     return category.strip()

# for index, row in categories_df.iterrows():
#     categories_df.at[index, 'Food Category'] = split_food_category(row['Food Category'])
    
# # Iterate through each row
# for index, row in consolidated_df.iterrows():
#     item_descrip = row['item_descrip']
    
#     # Check if "Food Category" is NaN
#     if pd.isna(row['Food Category']):
    
#         # Split the item_descrip into words
#         item_words = item_descrip.split()
        
#         # Find matches between the words in item_descrip and the Food Category column
#         matches = categories_df[categories_df['Food Category'].apply(lambda x: any(word in x.split() for word in item_words))]
        
#         # If there is a match, assign the corresponding values
#         if not matches.empty:
#             consolidated_df.at[index, 'Food Category'] = matches['Food Category'].iloc[0]
#             consolidated_df.at[index, 'Category ID'] = matches['Category ID'].iloc[0]

# # Print the updated DataFrame
# consolidated_df

In [12]:
filtered_df = consolidated_df[~consolidated_df["CategoryID"].isna()]
filtered_df

# filtered_df = a_df[~a_df["Category ID"].isna()]
# filtered_df


,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,item_descrip,Food Category
0,I-11706,41.0,Glry Side Fries 2023,1.0,ea,1.0,ea,N,glry side fries 2023,potatoes
2,I-14715,54.0,2022 Coleslaw mix,7.0,oz,1.0,Kg,N,2022 coleslaw mix,stimulants & spices misc.
4,I-13956,25.0,Sushi Rice,6.0,PORT,6.0,PORT,N,sushi rice,rice
6,I-16572,39.0,2022 Pickle onion,20.0,g,250.0,g,N,2022 pickle onion,onions and leeks
8,I-17039,31.0,2022 Grilled Peach Salsa2,30.0,g,1.0,g,N,2022 grilled peach salsa2,other fruits
...,...,...,...,...,...,...,...,...,...,...
94,I-15017,6.0,G21 Mac And Cheese,2.0,PORT,1.0,PORT,N,g21 mac and cheese,cheese
97,I-12339,25.0,rice vinegar,1.0,PORT,58.0,PORT,N,rice vinegar,rice
98,I-13398,54.0,BIB Mix Sprite,1.0,L,18.5,L,Y,bib mix sprite,stimulants & spices misc.
100,I-13736,38.0,2019 Ginger Garlic Paste,60.0,g,500.0,Kg,N,2019 ginger garlic paste,root vegetables


In [13]:
consolidated_df[(consolidated_df['CategoryID'] == 40.0) & (~consolidated_df['ItemId'].duplicated(keep=False))]

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,item_descrip,Food Category
88,I-13957,40.0,HR AVOCADO ROLL,1.0,PORT,1.0,PORT,N,hr avocado roll,other vegetables


In [14]:
unfiltered_df = consolidated_df[consolidated_df["CategoryID"].isna()]
unfiltered_df

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,item_descrip,Food Category
1,I-16780,NaN,2022 Vegan Caesar dressing,1.50,fl oz,1.00,fl oz,N,2022 vegan caesar dressing,NaN
3,I-15427,NaN,Vegan Chipotle Mayo,2.00,fl oz,1.00,L,N,vegan chipotle mayo,NaN
5,I-16221,NaN,2022 pickled jalapeno,40.00,g,250.00,g,N,2022 pickled jalapeno,NaN
7,I-6243,NaN,2019 Chipotle Aioli,4.00,fl oz,1.00,L,N,2019 chipotle aioli,NaN
9,I-17040,NaN,2022 Pickled Onion2,20.00,g,1.00,Kg,N,2022 pickled onion2,NaN
11,I-13414,NaN,linguini,1.00,PORT,1.00,PORT,N,linguini,NaN
15,I-16575,NaN,2022 Lime Aioli,30.00,g,1.00,g,N,2022 lime aioli,NaN
16,I-12266,NaN,G18 Parmesan Aioli,60.00,ml,1.00,L,N,g18 parmesan aioli,NaN
18,I-16787,NaN,2022 Blackning Spice,20.00,g,1.00,g,N,2022 blackning spice,NaN
20,I-11125,NaN,2022 Pico De Gallo,3.00,Tbsp,1.00,L,N,2022 pico de gallo,NaN


In [15]:
consolidated_df.drop(columns={'item_descrip'}, inplace = True)
consolidated_df

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-11706,41.0,Glry Side Fries 2023,1.0,ea,1.0,ea,N,potatoes
1,I-16780,NaN,2022 Vegan Caesar dressing,1.5,fl oz,1.0,fl oz,N,NaN
2,I-14715,54.0,2022 Coleslaw mix,7.0,oz,1.0,Kg,N,stimulants & spices misc.
3,I-15427,NaN,Vegan Chipotle Mayo,2.0,fl oz,1.0,L,N,NaN
4,I-13956,25.0,Sushi Rice,6.0,PORT,6.0,PORT,N,rice
...,...,...,...,...,...,...,...,...,...
98,I-13398,54.0,BIB Mix Sprite,1.0,L,18.5,L,Y,stimulants & spices misc.
99,I-13955,NaN,Sushi Su,1.0,L,50.0,L,N,NaN
100,I-13736,38.0,2019 Ginger Garlic Paste,60.0,g,500.0,Kg,N,root vegetables
101,I-18348,NaN,Gremolata 2022,5.0,g,1.0,g,N,NaN


In [16]:
# CHANGE THE ADRESS OF THE EXCEL FILE
consolidated_df.to_excel("C:/Users/smvan/CFFS-S23/CFFS-22-23/2023-12-22_New_Items.xlsx",index = None, header=True)

In [17]:
unfiltered_df = consolidated_df[consolidated_df["CategoryID"].isna()]
unfiltered_df.shape

(37, 9)